1. Choose a target variable from the dataset.

2. Determine whether the selected column's data can be used to solve a regression or classification problem.

3. Begin by preparing the dataset, separating it into features (X) and labels (Y).

4. Address missing data by either removing rows or filling in the gaps.

5. Utilize dummy hot encoding to transform non-numeric data into a numeric format.

6. Assemble the feature set (X) to include all relevant features, while the label set (Y) contains the values from the chosen column.

In [1]:
from google.colab import drive
drive.mount('Indicadores_municipales_sabana_DA')

Mounted at Indicadores_municipales_sabana_DA


In [16]:
import pandas as pd

# Load a CSV file into a DataFrame, specifying the encoding
df = pd.read_csv('Indicadores_municipales_sabana_DA.csv', encoding='latin1')


In [17]:
# Display the first rows of the DataFrame to verify that it has been loaded correctly
df.head()


ent         nom_ent  mun  clave_mun         nom_mun  pobtot_ajustada  \
0    1  Aguascalientes    1       1001  Aguascalientes           794304   
1    1  Aguascalientes    2       1002        Asientos            48592   
2    1  Aguascalientes    3       1003        Calvillo            53104   
3    1  Aguascalientes    4       1004           Cosío            14101   
4    1  Aguascalientes    5       1005     Jesús María           101379   

     pobreza  pobreza_e  pobreza_m    vul_car  ...  pobreza_alim_10  \
0  30.531104   2.264478  28.266627  27.983320  ...        11.805700   
1  67.111172   8.040704  59.070468  22.439389  ...        21.993299   
2  61.360527   7.241238  54.119289  29.428583  ...        19.266800   
3  52.800458   4.769001  48.031458  27.128568  ...        14.303200   
4  45.338512   6.084037  39.254475  26.262912  ...        15.085100   

   pobreza_cap_90  pobreza_cap_00  pobreza_cap_10  pobreza_patrim_90  \
0            20.4            12.7       18.474600               43.4   
1            39.9            29.0       30.980801               64.2   
2            39.5            33.1       28.259199               63.9   
3            35.2            21.0       22.386101               59.7   
4            36.6            22.6       22.139999               60.6   

   pobreza_patrim_00  pobreza_patrim_10  gini_90  gini_00   gini_10  
0               33.7          41.900398    0.473    0.425  0.422628  
1               48.9          59.175800    0.379    0.533  0.343879  
2               57.9          56.504902    0.414    0.465  0.386781  
3               40.1          51.164501    0.392    0.541  0.344984  
4               42.2          45.703899    0.391    0.469  0.458083  

[5 rows x 139 columns]

In [4]:
columns_to_exclude = ["ent", "nom_ent", "mun","nom_mun", "clave_mun","N_ic_rezedu"]
X = df.drop(columns_to_exclude, axis=1)
N_mun = df['nom_mun']
y = df["N_ic_rezedu"]

print("data already divided")

data already divided


data already divided

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

label_encoder = LabelEncoder()
X['gdo_rezsoc00'] = label_encoder.fit_transform(X['gdo_rezsoc00'])
X['gdo_rezsoc05'] = label_encoder.fit_transform(X['gdo_rezsoc05'])
X['gdo_rezsoc10'] = label_encoder.fit_transform(X['gdo_rezsoc10'])

imputer = SimpleImputer(strategy="mean")

X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X_imputed)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

In [7]:
import numpy as np
def calcular_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [18]:
def gradient_descent(X, y, X_test, y_test, learning_rate=0.001, epochs=10000):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)  # Initialize weights to zero
    bias = 0  # Initialize bias to zero

    training_errors = []  # Store errors during training
    test_errors = []  # Store errors on the test set

    for _ in range(epochs):
        # Calculate predictions
        predictions = np.dot(X, weights) + bias
        test_predictions = np.dot(X_test, weights) + bias

        # Calculate the error on the training and test sets
        training_error = calculate_error(y, predictions)
        test_error = calculate_error(y_test, test_predictions)

        # Store errors
        training_errors.append(training_error)
        test_errors.append(test_error)

        # Calculate gradients
        weight_gradients = -(2/num_samples) * np.dot(X.T, (y - predictions))
        bias_gradient = -(2/num_samples) * np.sum(y - predictions)

        # Update weights and bias using gradient descent
        weights -= learning_rate * weight_gradients
        bias -= learning_rate * bias_gradient
         # Intermediate prints

    return weights, bias, training_errors, test_errors


In [20]:
pesos, sesgo, errores_entrenamiento, errores_prueba = descenso_gradiente(X_train, Y_train, X_test, Y_test)
y_pred = np.dot(X_test, pesos) + sesgo

mse = calcular_error(Y_test, y_pred)

print(f"Error Cuadrático Medio: {mse}")

Error Cuadrático Medio: 12559199.935958525


In [21]:
import random

indice_aleatorio = random.choice(range(len(Y_test)))

nombre_municipio = N_mun.iloc[indice_aleatorio]
prediccion_desigualdad = y_pred[indice_aleatorio]
desigualdad_actual = Y_test.iloc[indice_aleatorio]

# Imprimir la información del municipio seleccionado al azar
print(f"For the municipality {nombre_municipio}, it is predicted that the number of people with social deprivation is {prediccion_desigualdad}. The real value of social deprivation is{desigualdad_actual}")


For the municipality Maravilla Tenejapa, it is predicted that the number of people with social deprivation is 2117.9397548769884. The real value of social deprivation is2772
